# Quasi Newton method

## Understanding the algorithm

La méthode quasi-Newton est une approche utilisée pour résoudre des problèmes d'optimisation continus sans contraintes, c'est-à-dire pour trouver les points où une fonction atteint un minimum local. Contrairement à la méthode de Newton classique qui requiert l'évaluation de la dérivée seconde (la hessienne), les méthodes quasi-Newton estiment la hessienne à chaque itération. Elles font partie d'une classe d'algorithmes dits de recherche de ligne.

Le terme "quasi-Newton" fait référence à une classe d'algorithmes d'optimisation qui visent à approximer les méthodes de Newton pour trouver les zéros ou les optima (minima ou maxima) d'une fonction. Les méthodes de Newton, également connues sous le nom de méthodes de Newton-Raphson, sont des approches efficaces pour trouver les solutions exactes de systèmes non linéaires et d'optimisation. Cependant, elles nécessitent le calcul de la dérivée seconde, ou hessienne, qui peut être très coûteux en termes de calculs, en particulier pour les problèmes de haute dimension.

Les méthodes de Newton classiques utilisent la dérivée première (le gradient pour les fonctions multivariables) et la dérivée seconde (la hessienne pour les fonctions multivariables) de la fonction objectif pour obtenir une formule itérative qui converge rapidement vers la solution, typiquement à une vitesse quadratique. Cela signifie que la méthode de Newton peut être très rapide pour trouver une racine ou un extremum, mais à chaque itération, elle nécessite le calcul et peut-être l'inversion de la matrice hessienne, ce qui est difficile et coûteux pour les grandes fonctions.

Pour surmonter cela, les méthodes quasi-Newton cherchent à obtenir une partie de l'efficacité des méthodes de Newton sans nécessiter le calcul explicite de la hessienne. Elles le font en construisant une approximation de la hessienne (ou de son inverse) à chaque étape, en utilisant des informations issues des gradients calculés lors des itérations précédentes. Cette approximation s'améliore à chaque itération, ce qui conduit éventuellement à une convergence vers la solution recherchée.

En d'autres termes, les méthodes quasi-Newton sont "quasi" parce qu'elles s'inspirent de la méthode de Newton en utilisant une approche similaire, mais ne nécessitent pas l'évaluation directe des dérivées secondes. Cela les rend plus pratiques pour les problèmes à grande échelle où l'évaluation de la hessienne est trop coûteuse. Elles offrent un bon compromis entre vitesse de convergence et coût computationnel, en faisant des méthodes de choix pour de nombreux problèmes d'optimisation en pratique.

## Usage examples

Les méthodes quasi-Newton sont souvent utilisées dans :

- L'optimisation de fonctions multivariées où le calcul de la hessienne est coûteux ou difficile.
- Les problèmes de machine learning, notamment l'entraînement des modèles où l'on doit minimiser une fonction de coût.
- Les problèmes d'ingénierie où des solutions optimales sont nécessaires mais la fonction objective est complexe.

## Strengths

1. Pas besoin de la hessienne exacte: Cela les rend utiles pour les grandes dimensions où calculer la hessienne exacte est impraticable.
2. Efficience en haute dimension: Elles peuvent être plus efficaces que la méthode de Newton pour les problèmes de grande taille.
3. Bonne convergence: Les méthodes quasi-Newton tendent à converger plus rapidement vers un minimum local que les méthodes de première ordre comme la descente la plus rapide.

## Weaknesses

1. Mémoire: Certaines implémentations peuvent consommer beaucoup de mémoire, ce qui peut être problématique avec des problèmes de très grande échelle.
2. Choix de l'initialisation: Le choix de la matrice initiale peut affecter la performance et la convergence de l'algorithme.
3. Convergence locale: Comme beaucoup de méthodes d'optimisation, elles garantissent seulement une convergence locale. Si la fonction a de multiples minimums, la solution trouvée dépend de la valeur initiale.

## Python demonstration

In [1]:
import numpy as np

# Fonction à minimiser
def func(x):
    return (x[0] - 1)**2 + (x[1] - 2.5)**2

# Gradient de la fonction
def grad_func(x):
    return np.array([2*(x[0] - 1), 2*(x[1] - 2.5)])

# Algorithme BFGS
def bfgs_quasi_newton_method(f, grad, x0, max_iter=100, tol=1e-5):
    n = len(x0)
    # Approximation initiale de la matrice hessienne inversée (identité)
    Hk = np.eye(n)
    
    xk = x0
    for _ in range(max_iter):
        # Calcul du gradient actuel
        gk = grad(xk)
        
        # Si le gradient est suffisamment petit, on considère avoir atteint le minimum
        if np.linalg.norm(gk) < tol:
            break
        
        # Direction de descente
        pk = -np.dot(Hk, gk)
        
        # Taille du pas - ici on utilise une recherche linéaire simple par backtracking
        alpha = 1
        while f(xk + alpha * pk) > f(xk) + 0.5 * alpha * np.dot(gk, pk):
            alpha *= 0.5
        
        # Mise à jour de la position
        xk_new = xk + alpha * pk
        gk_new = grad(xk_new)
        
        # Calcul du sk et yk pour la mise à jour de Hk
        sk = xk_new - xk
        yk = gk_new - gk
        
        # Mise à jour de la matrice hessienne inversée
        rho = 1.0 / (np.dot(yk, sk))
        I = np.eye(n)
        Hk = (I - rho * np.outer(sk, yk)) @ Hk @ (I - rho * np.outer(yk, sk)) + rho * np.outer(sk, sk)
        
        # Mise à jour de la position
        xk = xk_new
    
    return xk

# Point de départ
x0 = np.array([2.0, 0.0])

# Exécution de l'algorithme BFGS
xmin = bfgs_quasi_newton_method(func, grad_func, x0)

print("Le minimum estimé est à la position :", xmin)
print("La valeur minimale de la fonction est :", func(xmin))

Le minimum estimé est à la position : [1.  2.5]
La valeur minimale de la fonction est : 0.0


ans cet exemple, la fonction f est 
x^2 − 9, dont on sait que la racine est ±3. La dérivée df est 2x. 
On commence avec une estimation initiale x0 de 10. La tolérance tol est fixée à une petite valeur pour une grande précision, et max_iter limite le nombre d'itérations pour éviter une boucle infinie dans le cas où la méthode ne converge pas.

## Test de performance entre la methode de newton classique et une méthode quasi-Newton populaire comme BFGS

Pour comparer la performance de la méthode de Newton classique avec une méthode quasi-Newton populaire comme BFGS, nous pouvons tester les deux algorithmes sur un problème d'optimisation difficile. Un tel problème pourrait être la minimisation de la fonction de Rosenbrock, également connue sous le nom de fonction banane, qui est souvent utilisée pour tester les performances des algorithmes d'optimisation car elle a un minimum global dans un long et étroit vallon en forme de banane, ce qui rend la convergence difficile.

La fonction de Rosenbrock pour deux variables est définie comme suit: f(x,y)=(a−x) 2+b(y−x2)2 où typiquement a=1 et b=100. Le minimum global est atteint dans (x,y)=(a,a2), où f(x,y)=0.

Nous allons comparer la méthode de Newton nécessitant le gradient et la hessienne avec l'algorithme BFGS qui approxime la hessienne à chaque étape.

In [16]:
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import rosen, rosen_der
from scipy import optimize
import time

# Dimension du problème
n_dimensions = 100
maxiter = 2000
gtol = 1e-10

# Point de départ
x0 = np.random.rand(n_dimensions) * 10 - 5  # Initial guess

In [18]:
# Méthode quasi-Newton BFGS
start_time = time.time()
bfgs_result = minimize(rosen, x0, method='BFGS', jac=rosen_der, options={'gtol': gtol, 'maxiter': maxiter})
bfgs_time = time.time() - start_time

# Méthode de Newton-CG pour la minimisation multivariée
start_time = time.time()
newton_cg_result = minimize(optimize.rosen, x0, method='Newton-CG', jac=optimize.rosen_der, hess=optimize.rosen_hess, options={'xtol': gtol, 'maxiter': maxiter})
newton_cg_time = time.time() - start_time

In [20]:
# Vérifiez si la solution a convergé et imprimez les résultats
if bfgs_result.success:
    print(f"BFGS method result: {bfgs_result.x}, found in {bfgs_time:.4f} seconds")
else:
    print(f"BFGS method did not converge within the given iteration limit.")

print('\n')
# Vérifiez si la solution a convergé et imprimez les résultats
if newton_cg_result.success:
    print(f"Newton-CG method result: {newton_cg_result.x}, found in {newton_cg_time:.4f} seconds")
else:
    print(f"Newton-CG method did not converge within the given iteration limit or failed.")

BFGS method result: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.], found in 1.1409 seconds


Newton-CG method result: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.], found in 0.2184 seconds


Dans ce code, nous avons défini les fonctions pour la fonction de Rosenbrock, son gradient et sa hessienne, ainsi que deux fonctions pour exécuter les méthodes de Newton et BFGS. Nous exécutons ensuite les deux méthodes en partant du même point initial, et nous mesurons le temps d'exécution et le nombre d'itérations pour atteindre la convergence.

Il est important de noter que la méthode de Newton peut ne pas être bien adaptée à cette fonction sans une stratégie de recherche linéaire adaptative, car la méthode suppose que la fonction est quadratique autour du point actuel, ce qui n'est pas le cas ici. La méthode BFGS, en revanche, est plus flexible et souvent mieux adaptée à ce type de fonction complexe.

End of demonstration

---

## Practical optimization tools

- 5 examples

## Sources
- https://en.wikipedia.org/wiki/Tabu_search
- https://www.youtube.com/watch?v=saNk8h2KuVE
- https://www.youtube.com/watch?v=tIDhFPhrCbU